# 2 - Cleanup
Now that I have merged the downloaded data, I can start studying it to solve the issues thtat can by solved by deleting things.

In [39]:
import pandas as pd


# data = pd.read_csv('data\set_movies_sample_big.csv')
# data = pd.read_csv('data\imdb_full - 34k.csv')
data = pd.read_csv('data\imdb_full.csv')


In [40]:
data

,index,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,index.1,rating,genres.1,mpaa,description,votes
0,23130,tt0023526,movie,Stranger in Town,Stranger in Town,0,1932,\N,65,Drama,0,6.1,['drama'],TV-PG,"[""Ulysses Crickle owns a small town grocery st...",115.0
1,23131,tt0023527,movie,Strangers in Love,Strangers in Love,0,1932,\N,76,"Comedy,Drama",1,6.7,"['comedy', 'drama']",NaN,['_Strangers in Love_ (qv) is a 1932 American ...,144.0
2,23132,tt0023528,movie,Strangers of the Evening,Strangers of the Evening,0,1932,\N,70,"Comedy,Mystery",2,5.0,"['comedy', 'mystery']",Passed,['Bodies start mysteriously disappearing from ...,218.0
3,23133,tt0023529,movie,Street of Women,Street of Women,0,1932,\N,59,"Drama,Romance",3,6.0,"['drama', 'romance']",Passed,"[""A married architect, stuck in a loveless mar...",356.0
4,23134,tt0023530,movie,Spoiling the Game,Strich durch die Rechnung,0,1932,\N,99,Comedy,4,6.6,['comedy'],NaN,['Whoever of a group of eager bicycle racers f...,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61995,2995,tt1540764,movie,Anomaly,Anomaly,0,2010,\N,71,Sci-Fi,995,NaN,['sci-fi'],Not Rated,['A UFO crashes in the Bronx. Is it from anoth...,NaN
61996,2996,tt0228627,movie,Mí querido viejo,Mí querido viejo,0,1991,\N,\N,Drama,996,6.6,['drama'],NaN,NaN,58.0
61997,2997,tt0804466,movie,The Dork of the Rings,The Dork of the Rings,0,2006,\N,90,"Adventure,Comedy,Fantasy",997,4.3,"['adventure', 'comedy', 'fantasy']",Not Rated,['Frudo Buggins must deliver The One Ring of B...,142.0
61998,2998,tt6080158,movie,Not Alone,Not Alone,0,2016,\N,51,Documentary,998,6.1,['documentary'],TV-14,['Driven by a desire to understand why her bes...,192.0


## Columns
According to the definition in https://www.imdb.com/interfaces/ movies don't have an 'End Year', as that value is reserved for series. As such, this column has no use for me.

In [41]:
# data.endYear.value_counts()


In [42]:
cols_to_drop = []
cols_to_drop.append('endYear')

The column ``titleType`` has only movie as values, because I already filtered anything else at the beggining.

In [43]:
cols_to_drop.append('titleType')

The ``index`` column is here as a small mistake when merging the sources and can be dropped without issue. It happens 3 times, but they all will drop at the same time.
The ``tconst`` column, even though it will serve no purpose at modeling because it is just the ID in IMDb, will be kept in case I need to expand information in the future.

In [44]:
cols_to_drop.append('index')
cols_to_drop.append('index.1')

I have two columns of genders, but the second one has the same information but in a better format, so I will delete the other.

In [45]:
cols_to_drop.append('genres')

There are also two columns for names and they won't affect much, but still I will keep the names in English from ''primaryTitle''.

In [46]:
cols_to_drop.append('originalTitle')

Now it's time to drop some columns.

In [47]:
data = data.drop(cols_to_drop, 1)
data.head(1)

C:\Users\carlo\AppData\Local\Temp/ipykernel_1936/1662013735.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop(cols_to_drop, 1)


,tconst,primaryTitle,isAdult,startYear,runtimeMinutes,rating,genres.1,mpaa,description,votes
0,tt0023526,Stranger in Town,0,1932,65,6.1,['drama'],TV-PG,"[""Ulysses Crickle owns a small town grocery st...",115.0


## NaNs

In [48]:
data.isna().sum()/len(data)

tconst            0.000000
primaryTitle      0.000000
isAdult           0.000000
startYear         0.000000
runtimeMinutes    0.000000
rating            0.255145
genres.1          0.010355
mpaa              0.666306
description       0.250258
votes             0.255145
dtype: float64

From this overview I can get some conclussions:  
- The first columns are all populated because the come from an official database  
- the last 2 columns appear as filled with errors due to a problem with the scraper, I will try to solve it later.

### ``rating``
This column is going to be one of my targets, so I need to keep the actual data and remove the rest.

In [49]:
data = data.dropna(subset=['rating'])

### ``mpaa``
The age rating has a list of defined values: 
* **G – General Audiences**
* **PG – Parental Guidance Suggested**
* **PG-13 – Parents Strongly Cautioned**
* **R – Restricted**
* **NC-17 – Adults Only**


In [50]:
data.mpaa.value_counts(dropna=False)

NaN          26170
Approved      6188
Not Rated     4329
Passed        3662
R             1831
PG             706
TV-PG          538
Unrated        532
G              427
PG-13          363
TV-14          297
X              283
GP             227
TV-G           222
TV-MA          217
M               92
M/PG            60
TV-Y7           13
NC-17           10
TV-Y7-FV         5
TV-Y             4
Open             2
TV-13            2
E                1
Name: mpaa, dtype: int64


For now, I will assign the empty values to the most common appearance, that is Not Rated.
The rest of the changes will come later.

In [51]:
data.mpaa.fillna('Not Rated', inplace=True)
# data.mpaa.value_counts(dropna=False)

### ``description``
As the most important column, I have no use for rows with no information in it, so I need to remove them.

In [52]:
data = data.dropna(subset=['description'])

Lastly, the columns that come from Rotten Tomatoes have to be deleted, as I have lost access to the web.

In [53]:
# data = data.drop(['rt_critics_score', 'rt_audience_score'], 1)
# data.head(1)

In [54]:
# data.to_csv('data\movies_clean.csv', index=False)
data.to_csv('data\sample_clean.csv', index=False)

In [55]:
data.isna().sum()/len(data)

tconst            0.0
primaryTitle      0.0
isAdult           0.0
startYear         0.0
runtimeMinutes    0.0
rating            0.0
genres.1          0.0
mpaa              0.0
description       0.0
votes             0.0
dtype: float64